In [2]:
# <게임 이름 스크래핑>
import csv
import MySQLdb  # mysql, mariadb 연동모듈
from datetime import datetime
import requests
from bs4 import BeautifulSoup

# 데이터베이스 연결 설정
conn = MySQLdb.connect(host="localhost", port=3306, db='gamelist', user='root', password='12345', charset='utf8')
cursor = conn.cursor()

# 웹 페이지에서 데이터 스크랩
url = "https://play.google.com/store/games?device=phone&hl=ko"
res = requests.get(url)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

# 'find_all'을 사용하여 원하는 요소를 찾습니다.
games = soup.find_all("span", class_="sT93pb DdYX5 OnEJge")

# 데이터베이스에 게임 이름 저장
for game in games:
    game_name = game.text.strip()  # 게임 이름 추출
    query = "INSERT INTO games (game_name) VALUES (%s)"
    cursor.execute(query, (game_name,))  # 튜플 형태로 매개변수를 전달

# 변경사항을 데이터베이스에 커밋
conn.commit()

# 데이터베이스 연결 종료
cursor.close()
conn.close()

In [4]:
# 투표 및 랭킹 DB저장
import requests
from bs4 import BeautifulSoup
import MySQLdb  # MySQL, MariaDB 연동 모듈
import random
import time
import schedule
from datetime import datetime

# 데이터베이스 연결 설정
conn = MySQLdb.connect(host="localhost", port=3306, db='gamelist', user='root', password='12345', charset='utf8')
cursor = conn.cursor()

# 게임 이름 후보자 가져오기
def get_candidates():
    query = "SELECT id, game_name FROM games"
    cursor.execute(query)
    return cursor.fetchall()

# 투표 함수
def cast_vote(candidate_id):
    query = "UPDATE games SET game_vote = game_vote + 1 WHERE id = %s"
    cursor.execute(query, (candidate_id,))

# 현재 투표 결과를 저장하는 함수
def save_current_results():
    timestamp = datetime.now()
    query = """
    SELECT id, game_vote
    FROM games
    ORDER BY game_vote DESC
    """
    cursor.execute(query)
    results = cursor.fetchall()
    
    for rank, (game_id, vote_count) in enumerate(results, 1):
        update_query = """
        UPDATE games SET `game_rank` = %s WHERE id = %s
        """
        cursor.execute(update_query, (rank, game_id))
    
    conn.commit()
    print(f"현재 결과 저장됨: {timestamp}")

# 결과 보기 함수
def show_results():
    query = """
    SELECT game_name, game_vote
    FROM games
    ORDER BY game_vote DESC
    LIMIT 50
    """
    cursor.execute(query)
    results = cursor.fetchall()
    print("\n상위 50개 게임 투표 결과:")
    for idx, result in enumerate(results, 1):
        print(f"{idx}위: {result[0]}: {result[1]} 명 투표")

# 투표 기록 초기화
def reset_votes():
    query = "UPDATE games SET game_vote = 0"
    cursor.execute(query)
    conn.commit()

# 1,000명의 사용자 투표 시뮬레이션
def simulate_voting():
    candidates = get_candidates()
    candidate_ids = [candidate[0] for candidate in candidates]

    for _ in range(1000):
        candidate_id = random.choice(candidate_ids)
        cast_vote(candidate_id)
    
    conn.commit()  # 모든 투표를 한 번에 커밋

# 1분마다 실행되는 작업
def job():
    reset_votes()
    simulate_voting()
    show_results()

# 1분마다 실행되는 작업 (결과 저장)
def daily_job():
    save_current_results()

# 스케줄 설정
# schedule.every(1).minutes.do(job)
# schedule.every(1).minutes.do(daily_job)

schedule.every(10).seconds.do(job)  # 10초마다 job() 함수 실행
schedule.every(10).seconds.do(daily_job)  # 10초마다 daily_job() 함수 실행

# 초기 실행
job()

while True:
    schedule.run_pending()
    time.sleep(1)

# 데이터베이스 연결 종료
cursor.close()
conn.close()


상위 50개 게임 투표 결과:
1위: 드래곤 윙스 - 판타지 슈터: 20 명 투표
2위: Dye Hard - Color War: 20 명 투표
3위: 꿈의 집 (Homescapes): 19 명 투표
4위: 마작 솔리테어 클래식 게임: 18 명 투표
5위: 앵그리버드2: 18 명 투표
6위: Cooking Story: Chef's Journey: 17 명 투표
7위: Candy Crush Saga: 17 명 투표
8위: Bingo Blitz™️ - Bingo Games: 17 명 투표
9위: Hedgies: Farming & Building: 16 명 투표
10위: 앵그리버드2: 16 명 투표
11위: 로드 모바일:라스트 파이터: 16 명 투표
12위: Find Stuff in America: 16 명 투표
13위: Idle Town Master - Pixel Game: 16 명 투표
14위: 언더 서바이벌 : 어드벤쳐&시뮬레이션: 16 명 투표
15위: 소울웨폰 키우기: 16 명 투표
16위: 마이토킹행크: 아일랜드: 16 명 투표
17위: DRAGON BALL LEGENDS 드래곤 볼 레전즈: 15 명 투표
18위: 아우터플레인 (OUTERPLANE): 15 명 투표
19위: 클래시 로얄: 15 명 투표
20위: 라스트 쉘터: 서바이벌: 15 명 투표
21위: Age of Apes: 원숭이 대전: 15 명 투표
22위: 둠스데이: 라스트 서바이버: 15 명 투표
23위: Chaos Galaxy - 우주 사격수: 14 명 투표
24위: 아스팔트 9: 레전드: 14 명 투표
25위: Dungeons of Souls: 14 명 투표
26위: Amikin Survival: 14 명 투표
27위: Sort Match:3D Goods Master: 14 명 투표
28위: Among Us: 14 명 투표
29위: 히어로 어셈블: 방치형 액션 RPG: 13 명 투표
30위: 헝그리 샤크 에볼루션: 최강 상어 먹방 서바이벌 게임: 13 명 투표
31위: Mini Heroe

KeyboardInterrupt: 

In [ ]:
<게임 이름 스크래핑>
import csv
import MySQLdb  # mysql, mariadb 연동모듈
from datetime import datetime
import requests
from bs4 import BeautifulSoup

# 데이터베이스 연결 설정
conn = MySQLdb.connect(host="localhost", port=3306, db='gamelist', user='root', password='12345', charset='utf8')
cursor = conn.cursor()

# 웹 페이지에서 데이터 스크랩
url = "https://play.google.com/store/games?device=phone&hl=ko"
res = requests.get(url)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

# 'find_all'을 사용하여 원하는 요소를 찾습니다.
games = soup.find_all("span", class_="sT93pb DdYX5 OnEJge")

# 데이터베이스에 게임 이름 저장
for game in games:
    game_name = game.text.strip()  # 게임 이름 추출
    query = "INSERT INTO games (name) VALUES (%s)"
    cursor.execute(query, (game_name,))  # 튜플 형태로 매개변수를 전달

# 변경사항을 데이터베이스에 커밋
conn.commit()

# 데이터베이스 연결 종료
cursor.close()
conn.close()

In [ ]:
<게임 1분마다 투표, 순위>
import requests
from bs4 import BeautifulSoup
import MySQLdb  # MySQL, MariaDB 연동 모듈
import random
import time
import schedule
from datetime import datetime

# 데이터베이스 연결 설정
conn = MySQLdb.connect(host="localhost", port=3306, db='gamelist', user='root', password='12345', charset='utf8')
cursor = conn.cursor()

# 게임 이름 후보자 가져오기
def get_candidates():
    query = "SELECT id, name FROM games"
    cursor.execute(query)
    return cursor.fetchall()

# 투표 함수
def cast_vote(candidate_id):
    query = "UPDATE games SET vote = vote + 1 WHERE id = %s"
    cursor.execute(query, (candidate_id,))

# 현재 투표 결과를 저장하는 함수
def save_current_results():
    timestamp = datetime.now()
    query = """
    SELECT id, vote
    FROM games
    ORDER BY vote DESC
    """
    cursor.execute(query)
    results = cursor.fetchall()
    
    for rank, (game_id, vote_count) in enumerate(results, 1):
        update_query = """
        UPDATE games SET `rank` = %s WHERE id = %s
        """
        cursor.execute(update_query, (rank, game_id))
    
    conn.commit()
    print(f"현재 결과 저장됨: {timestamp}")

# 결과 보기 함수
def show_results():
    query = """
    SELECT name, vote
    FROM games
    ORDER BY vote DESC
    LIMIT 50
    """
    cursor.execute(query)
    results = cursor.fetchall()
    print("\n상위 50개 게임 투표 결과:")
    for idx, result in enumerate(results, 1):
        print(f"{idx}위: {result[0]}: {result[1]} 명 투표")

# 투표 기록 초기화
def reset_votes():
    query = "UPDATE games SET vote = 0"
    cursor.execute(query)
    conn.commit()

# 1,000명의 사용자 투표 시뮬레이션
def simulate_voting():
    candidates = get_candidates()
    candidate_ids = [candidate[0] for candidate in candidates]

    for _ in range(1000):
        candidate_id = random.choice(candidate_ids)
        cast_vote(candidate_id)
    
    conn.commit()  # 모든 투표를 한 번에 커밋

# 1분마다 실행되는 작업
def job():
    reset_votes()
    simulate_voting()
    show_results()

# 1분마다 실행되는 작업 (결과 저장)
def daily_job():
    save_current_results()

# 스케줄 설정
schedule.every(1).minutes.do(job)
schedule.every(1).minutes.do(daily_job)

# 초기 실행
job()

while True:
    schedule.run_pending()
    time.sleep(1)

# 데이터베이스 연결 종료
cursor.close()
conn.close()







<이미지 스크래핑 저장>


<이미지 스크래핑 >
import requests
from bs4 import BeautifulSoup

url = "https://play.google.com/store/games?device=phone&hl=ko"
res = requests.get(url)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

# 게임 카드 요소를 찾습니다.
game_cards = soup.find_all("div", class_="j2FCNc")

# 게임 이름과 이미지 URL 추출
for idx, game_card in enumerate(game_cards, start=1):
    # 게임 이름 찾기
    game_name_tag = game_card.find("span", class_="DdYX5")
    if game_name_tag:
        game_name = game_name_tag.text
    else:
        continue  # 게임 이름을 찾을 수 없으면 생략
    
    # 게임 이미지 URL 찾기
    game_image_tag = game_card.find("img", class_="T75of stzEZd")
    if game_image_tag:
        game_image = game_image_tag["src"]
    else:
        continue  # 게임 이미지를 찾을 수 없으면 생략
    
    print(f"{idx}. 게임 이름: {game_name}, 이미지 URL: {game_image}")

In [3]:
# 게임 이미지 저장
import requests
from bs4 import BeautifulSoup
import MySQLdb

# 데이터베이스 연결 설정
conn = MySQLdb.connect(host="localhost", port=3306, db='gamelist', user='root', password='12345', charset='utf8')
cursor = conn.cursor()

url = "https://play.google.com/store/games?device=phone&hl=ko"
res = requests.get(url)
res.raise_for_status()

soup = BeautifulSoup(res.text, "lxml")

# 게임 카드 요소를 찾습니다.
game_cards = soup.find_all("div", class_="j2FCNc")

# 게임 이름과 이미지 URL 추출 및 데이터베이스에 저장
for idx, game_card in enumerate(game_cards, start=1):
    # 게임 이름 찾기
    game_name_tag = game_card.find("span", class_="DdYX5")
    if game_name_tag:
        game_name = game_name_tag.text
    else:
        continue  # 게임 이름을 찾을 수 없으면 생략
    
    # 게임 이미지 URL 찾기
    game_image_tag = game_card.find("img", class_="T75of stzEZd")
    if game_image_tag:
        game_image = game_image_tag["src"]
    else:
        continue  # 게임 이미지를 찾을 수 없으면 생략
    
    # 데이터베이스 업데이트
    sql = "UPDATE games SET game_image = %s WHERE game_name = %s"
    cursor.execute(sql, (game_image, game_name))
    conn.commit()
    
    print(f"{idx}. 게임 이름: {game_name}, 이미지 URL: {game_image}")

# 연결 종료
cursor.close()
conn.close()

10. 게임 이름: 클래시 오브 클랜, 이미지 URL: https://play-lh.googleusercontent.com/LByrur1mTmPeNr0ljI-uAUcct1rzmTve5Esau1SwoAzjBXQUby6uHIfHbF9TAT51mgHm=s64
11. 게임 이름: 꿈의 마을 (Township), 이미지 URL: https://play-lh.googleusercontent.com/9KAPlokfzeEFb34Mwm7juoiC5-yjpf4JlRLguuAgn5fV7JcTECwJnu6guLxbaZBWDA=s64
12. 게임 이름: Free Fire MAX, 이미지 URL: https://play-lh.googleusercontent.com/Y35F4HY9dw5_tf5Jjc0r8vWCvFWU7yIaXGjNRKnMOtYsmUNiy-LziZlDWbhpKYl1PBHd=s64
13. 게임 이름: Evony: The King's Return, 이미지 URL: https://play-lh.googleusercontent.com/-kxC4InQ01s06oRPVdInILaIitVUs5QD4IFGAFeXWTR62gAF-BPJuhXZi_cPvVB13oE=s64
14. 게임 이름: Klondike Adventures: 농장게임, 이미지 URL: https://play-lh.googleusercontent.com/BQjxTr1_93OrGMBio3r_YtFFeLUFL2O2NMmtFaUIJ5sYh1cOtvy61b6S8aIjWTySHls=s64
15. 게임 이름: 클래시 로얄, 이미지 URL: https://play-lh.googleusercontent.com/Uew-Z0cVBLaJF4p9u0HEIXiLU4HKlxJc7z9BN_ypMO4HORIZknsyQHcaJrm7gr4pfm4=s64
16. 게임 이름: DRAGON BALL LEGENDS 드래곤 볼 레전즈, 이미지 URL: https://play-lh.googleusercontent.com/sXGEcbamdFwjikD-nk8KaEGd8